Notebook responsável para realizar transformações no dataframe de contratos. Ele está sendo usado apenas para testes, código completos serão enviados para o arquivo de transform.py

Carregamento do arquivo bruto de contratos 👇

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime

# Configura o caminho para os arquivos brutos
raw_data_dir = Path('..') / 'data' / 'raw'

# Localiza o arquivo de contratos mais recente (pelo formato de data no nome do arquivo)
contratos_files = list(raw_data_dir.glob('contratos_amostra_*.csv'))

if contratos_files:
    # Pega o arquivo mais recente baseado na data de modificação
    latest_contratos_file = max(contratos_files, key=lambda x: x.stat().st_mtime)

    # Verificação (para debug)
    print(f'Arquivo de contratos encontrado: {latest_contratos_file}')
    print(f'O arquivo existe? {latest_contratos_file.exists()}')

    # Carrega o Dataframe
    df = pd.read_csv(latest_contratos_file, encoding='utf-8', low_memory=False)

    # Visualiza dimensões e primeiras linhas
    print(f'\nDataframe carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')
    print(f'Primeira 5 colunas: {', '.join(df.columns[:5])}')

else:
    print('ERRO: Nenhum arquivo de contratos encontrado em data/raw')
    print(f'Diretório verificado: {raw_data_dir.resolver()}')
    print(f'O diretório existe? {raw_data_dir.exists()}')
    print('Arquivos no diretório:', list(raw_data_dir.glob('*')))

Arquivo de contratos encontrado: ..\data\raw\contratos_amostra_2025-08-29.csv
O arquivo existe? True

Dataframe carregado: 20000 linhas, 39 colunas
Primeira 5 colunas: codigoOrgao, nomeOrgao, codigoUnidadeGestora, nomeUnidadeGestora, codigoUnidadeGestoraOrigemContrato


### ANÁLISE GERAL DO DATAFRAME

In [2]:
df.head(10)

,codigoOrgao,nomeOrgao,codigoUnidadeGestora,nomeUnidadeGestora,codigoUnidadeGestoraOrigemContrato,nomeUnidadeGestoraOrigemContrato,receitaDespesa,numeroContrato,codigoUnidadeRealizadoraCompra,nomeUnidadeRealizadoraCompra,...,valorAcumulado,totalDespesasAcessorias,dataHoraInclusao,numeroControlePncpContrato,idCompra,dataHoraExclusao,contratoExcluido,unidadesRequisitantes,data_inicio,trimestre
0,22000,MINISTERIO DA AGRICULTURA E PECUARIA,130080,SUPERINT.DE AGRICULTURA E PECUARIA - SFA/GO,130080,SUPERINT.DE AGRICULTURA E PECUARIA - SFA/GO,D,00009/2023,130080.0,SUPERINT.DE AGRICULTURA E PECUARIA - SFA/GO,...,2651257.96,NaN,2023-12-26T08:38:02,NaN,13008005000042023,NaN,False,NaN,2024-01-01,1
1,36000,MINISTERIO DA SAUDE,257003,INSTITUTO EVANDRO CHAGAS,257003,INSTITUTO EVANDRO CHAGAS,D,00020/2024,257003.0,INSTITUTO EVANDRO CHAGAS,...,NaN,NaN,2024-01-26T15:29:42,NaN,25700305000512023,NaN,False,NaN,2024-01-26,1
2,30202,FUNDACAO NACIONAL DOS POVOS INDIGENAS,194042,COORD. REG. RIBEIRAO CASCALHEIRA/MT,194042,COORD. REG. RIBEIRAO CASCALHEIRA/MT,D,2024NE000016,194042.0,COORD. REG. RIBEIRAO CASCALHEIRA/MT,...,NaN,NaN,2025-01-15T11:21:03,NaN,19404207000022023,NaN,False,NaN,2024-02-09,1
3,12000,JUSTICA FEDERAL,90027,SECRETARIA DO T.R.F. DA 1A. REGIAO,90027,SECRETARIA DO T.R.F. DA 1A. REGIAO,D,00081/2023,90027.0,SECRETARIA DO T.R.F. DA 1A. REGIAO,...,363534.30,NaN,2024-09-06T15:19:54,NaN,09002705000552023,NaN,False,NaN,2024-01-02,1
4,15000,JUSTICA DO TRABALHO,80001,TRIBUNAL SUPERIOR DO TRABALHO,80001,TRIBUNAL SUPERIOR DO TRABALHO,D,2024NE000506,80001.0,TRIBUNAL SUPERIOR DO TRABALHO,...,37450.00,NaN,2024-03-07T12:22:32,NaN,08000106000142024,NaN,False,CMAP,2024-03-07,1
5,26434,"INST.FED.DE EDUC.,CIENC.E TEC.FLUMINENSE",158139,"INST.FED.DE EDUC.,CIENC.E TEC.FLUMINENSE",158139,"INST.FED.DE EDUC.,CIENC.E TEC.FLUMINENSE",D,00019/2023,158139.0,"INST.FED.DE EDUC.,CIENC.E TEC.FLUMINENSE",...,328102.32,NaN,2023-12-28T11:56:07,NaN,15813905000682023,NaN,False,NaN,2024-01-02,1
6,41231,AGENCIA NACIONAL DE TELECOMUNICACOES,413001,AGENCIA NACIONAL DE TELECOMUNICACOES-SEDE,413001,AGENCIA NACIONAL DE TELECOMUNICACOES-SEDE,D,00026/2024,413001.0,AGENCIA NACIONAL DE TELECOMUNICACOES-SEDE,...,64540.00,NaN,2024-03-11T06:55:29,NaN,41300107000282024,NaN,False,AFPE,2024-03-08,1
7,95448,PMSP - SECRETARIA MUNICIPAL DA FAZENDA,925011,PMSP - SECRETARIA MUNICIPAL DA FAZENDA,925011,PMSP - SECRETARIA MUNICIPAL DA FAZENDA,D,2024NE033459,925011.0,PMSP - SECRETARIA MUNICIPAL DA FAZENDA,...,10368.72,NaN,2024-03-19T17:18:08,NaN,92501106900012024,NaN,False,NaN,2024-03-08,1
8,26443,EMPRESA BRASILEIRA DE SERVICOS HOSPITALARES,155908,HOSPITAL UNIV. DR. MIGUEL RIET CORREA JR.,155908,HOSPITAL UNIV. DR. MIGUEL RIET CORREA JR.,D,00004/2024,155908.0,HOSPITAL UNIV. DR. MIGUEL RIET CORREA JR.,...,5469.48,NaN,2024-02-06T16:17:26,NaN,15590805000482023,NaN,False,NaN,2024-02-08,1
9,36201,FUNDACAO OSWALDO CRUZ,254447,INST NACIONAL DE SAUDE FERNANDES FIGUEIRA,254447,INST NACIONAL DE SAUDE FERNANDES FIGUEIRA,D,00004/2024,254447.0,INST NACIONAL DE SAUDE FERNANDES FIGUEIRA,...,2760.00,NaN,2024-01-18T11:38:05,NaN,25444705000492023,NaN,False,254447,2024-01-17,1


Verificação de Nulos por quantidade e porcentagem 👇

In [3]:
# Calcula contagem e percentual de nulos em cada coluna
nulos = df.isnull().sum()
percentual = (df.isnull().sum() / len(df) * 100).round(2)

# Cria um Dataframe simples apenas com colunas que têm pelo menos um valor nulo
colunas_com_nulos = pd.DataFrame({
    'Valores Nulos': nulos[nulos >0],
    'Percentual (%)': percentual[nulos > 0]
}).sort_values('Percentual (%)', ascending=False)

# Exibe o resultado
colunas_com_nulos

,Valores Nulos,Percentual (%)
dataHoraExclusao,20000,100.00
numeroControlePncpContrato,20000,100.00
totalDespesasAcessorias,19999,100.00
nomeSubcategoria,19416,97.08
codigoSubcategoria,19416,97.08
unidadesRequisitantes,17944,89.72
valorAcumulado,11499,57.50
informacoesComplementares,9462,47.31
codigoTipo,7007,35.03
nomeTipo,7007,35.03


### ANÁLISE E TRATAMENTO DAS COLUNAS NULAS

**Análise da coluna `nomeSubcategoria`** \
*OBS: também inclui `codigoSubcategoria`*
- Total de nulos: 95,52%
- Valores não nulos mais comuns: Nota Fiscal Eletrônica, Quinzenal, Diária, etc
- Quantidade de nulos por `nomeCategoria`: maioria acima de 95%

Conclusão: coluna com altíssima proporção de nulos e baixa utilidade -> <span style='color:red; font-weight:bold;'>descartar</span>

In [ ]:
import re
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeSubcategoria' in df.columns:
    # 1. Verifica porcentagem de nulos de 'nomeSubcategoria'
    nulos_sub = df['nomeSubcategoria'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_sub / total * 100).round(2)
    print(f"\nTotal de nulos: {nulos_sub} ({percentual_nulos}%)")

    # 2. Verifica valores não nulos mais comuns
    # Limpeza de valores não nulos (remove URLs)
    df['nomeSubcategoria'] = df['nomeSubcategoria'].apply(lambda x: x if pd.isnull(x) or not re.match(r'^https?://', str(x)) else None)
    if df['nomeSubcategoria'].notnull().any():
        valores_comuns = df['nomeSubcategoria'].value_counts().head(10)
        print("\nValores mais comuns que aparecem em 'nomeSubcategoria':")
        print(valores_comuns)

    # 3. Verifica relação com coluna 'nomeCategoria'
    if 'nomeCategoria' in df.columns:
        nulos_por_categoria = df.groupby('nomeCategoria')['nomeSubcategoria'].apply(lambda x: x.isnull().mean() * 100).round(2)
        print("\nPercentual de nulos em 'nomeSubcategoria' por 'nomeCategoria':")
        print(nulos_por_categoria.sort_values(ascending=False))

    # 4. Conclusão baseada na análise
    print("\nConclusão baseada na análise:")
    if percentual_nulos > LIMITE_NULOS:
        print(f"- Mais de {LIMITE_NULOS}% de nulos → candidata forte à exclusão")
    else:
        print("- Percentual de nulos abaixo do limite, avaliar manutenção")

---

**Análise da coluna `unidadesRequisitantes`** \
*OBS: na documentação da API, essa coluna está com o nome um pouco diferente e possui uma coluna auxiliar de código de identificação. As duas colunas são `codigoUnidadeRequisitante` e `nomeUnidadeRequisitante` Não se sabe o motivo dessa mudança e da exclusão da coluna de código, mas deve ser levado em conta na hora de ler a documentação do endpoint de contratos da API*
- Total de nulos: 90,20%
- Valores não nulos mais comuns: EGESP, 090172, CENTRO DE TRABALHO E EDUCAÇÃO, SGM/CAF/DAP...
- Órgãos com unidade requisitante: 122 de 299 (40,8%)
- Órgãos mais associados: JUSTICA ELEITORAL, ESP-SECRETARIA DA SAUDE, ESP-SECRETARIA DA FAZENDA E PLANEJAMENTO, ESP-SECRETARIA ADMINISTRACAO PENITENCIARIA...

Conclusão: coluna com proporção muito alta de nulos e presente em menos da metade dos órgãos -> <span style='color:red; font-weight:bold;'>descartar</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'unidadesRequisitantes' in df.columns:
    # 1. Verifica porcentagem de nulos de 'unidadesRequisitantes'
    nulos_sub = df['unidadesRequisitantes'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_sub / total * 100).round(2)
    print(f"\nTotal de nulos: {nulos_sub} ({percentual_nulos}%)")

    # 2. Verifica valores não nulos mais comuns
    if df['unidadesRequisitantes'].notnull().any():
        valores_comuns = df['unidadesRequisitantes'].value_counts().head(10)
        print("\nValores mais comuns que aparecem em 'unidadesRequisitantes':")
        print(valores_comuns)

    # 3. Verifica relação com coluna 'nomeOrgao'
    if 'nomeOrgao' in df.columns:
        df_unidades = df[df['unidadesRequisitantes'].notnull()]

        # 3.1 verifica quais órgãos mais aparecem quando unidadesRequisitantes não está nula
        orgaos_por_unidade = df_unidades.groupby('nomeOrgao')['unidadesRequisitantes'].count().sort_values(ascending=False)
        print("\nÓrgãos com mais unidades requisitantes:")
        print(orgaos_por_unidade)

        # 3.2 verifica quantos órgão diferentes têm algum registro de unidadesRequisitantes
        total_orgaos = df['nomeOrgao'].nunique()
        num_orgaos_diferentes = df_unidades['nomeOrgao'].nunique()
        print(f"\nTotal de órgãos: {total_orgaos}")
        print(f"Órgãos com unidades requisitantes: {num_orgaos_diferentes} de {total_orgaos} ({num_orgaos_diferentes/total_orgaos:.1%})")

    # 4. Conclusão baseada na análise
    print("\nConclusão baseada na análise:")
    if percentual_nulos > LIMITE_NULOS:
        print(f"- Mais de {LIMITE_NULOS}% de nulos → candidata forte à exclusão")
    else:
        print("- Percentual de nulos abaixo do limite, avaliar manutenção")

**Análise da coluna `valorAcumulado`**  
- Total de nulos: 52,39%  
- Comparação com `valorGlobal`: alguns registros apresentam diferenças grandes, incluindo valores negativos e outliers muito altos, provavelmente por ajustes ou juros.  

Observações:
- Não passa muita confiança nos valores. Em alguns casos o valorAcumulado é maior que o valor total do contrato. Isso pode estar relacionado a juros ou reajustes, mas a API não fornece metadados suficientes para confirmar.

Conclusão: coluna com quantidade alta de nulos e valores inconsistentes. Em alguns casos, o `valorAcumulado` supera o `valorGlobal`, mas não há metadados que expliquem essa diferença (ex.: juros ou reajustes). Para análises gerais, `valorGlobal` é suficiente → <span style='color:red; font-weight:bold;'>descartar</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'valorAcumulado' in df.columns:
    # 1. Verifica porcentagem de nulos de 'valorAcumulado'
    nulos_sub = df['valorAcumulado'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_sub / total * 100).round(2)
    print(f"\nTotal de nulos: {nulos_sub} ({percentual_nulos}%)")

    # 2. Verifica quantos registros têm 'valorAcumulado' diferente de 'valorGlobal'
    if 'valorGlobal' in df.columns:
        valores_diferentes = df[df['valorAcumulado'].notnull() & (df['valorAcumulado'] != df['valorGlobal'])]
        print(f"Registros com valorAcumulado diferente de valorGlobal: {len(valores_diferentes)}")

        # 2.1 Verifica discrepância entre as duas colunas
        diff = df[df['valorAcumulado'].notnull() & (df['valorAcumulado'] != df['valorGlobal'])].copy()
        diff['diferenca'] = diff['valorAcumulado'] - diff['valorGlobal']
        display(diff[['valorGlobal','valorAcumulado','diferenca']].describe())

**Análise da coluna `nomeTipo`** \
*OBS: também inclui `codigoTipo`*
- Total de nulos: 41,64%
- Valores não nulos mais comuns: Empenho, Outros, Acordo de Cooperação Técnica (ACT), Carta Contrato

Observações:
- A coluna `nomeTipo` é bem redundante, já que mais de 90% dos registros não nulos são Empenho. Além disso, a ausência de preenchimento está possivelmente relacionada a certas categorias e modalidades de compra, indicando que o campo não é usado de forma consistente.”

Conclusão: A coluna apresenta muitos nulos e, quando preenchida, quase sempre indica ‘Empenho’. Por isso, acaba não oferecendo informações adicionais muito relevantes → <span style='color:red; font-weight:bold;'>descartar</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeTipo' in df.columns:
    # 1. Verifica porcentagem de nulos de 'nomeTipo'
    nulos_tipo = df['nomeTipo'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_tipo / total * 100).round(2)
    print(f"\nTotal de nulos em nomeTipo: {nulos_tipo} ({percentual_nulos}%)")

    # 2. Verifica valores não nulos mais comuns
    if df['nomeTipo'].notnull().any():
        valores_comuns = df['nomeTipo'].value_counts().head(10)
        print(f"Valores não nulos mais comuns em nomeTipo:\n{valores_comuns}")

    # 3 Verifica relação com 'nomeCategoria' usando crosstab (com normalização)
    if 'nomeCategoria' in df.columns:
        print("\n--- Relação entre nomeTipo e nomeCategoria ---")
        
        # Porcentagem de nulos por categoria
        nulos_por_categoria = df.groupby('nomeCategoria')['nomeTipo'].apply(lambda x: (x.isnull().sum() / len(x) * 100)).round(2).sort_values(ascending=False)
        print("Porcentagem de nulos por categoria:")
        print(nulos_por_categoria.head(10))
        
        # Contagem absoluta de nulos por categoria
        contagem_nulos = df[df['nomeTipo'].isnull()].groupby('nomeCategoria').size().sort_values(ascending=False)
        print("\nQuantidade de nulos por categoria (top 10):")
        print(contagem_nulos.head(10))

    # 4. Verifica relação com 'nomeModalidadeCompra' (se existir)
    if 'nomeModalidadeCompra' in df.columns:
        print("\n--- Relação entre nomeTipo e nomeModalidadeCompra ---")

        # Porcentagem de nulos por modalidade
        nulos_por_modalidade = df.groupby('nomeModalidadeCompra')['nomeTipo'].apply(lambda x: (x.isnull().sum() / len(x) * 100)).round(2).sort_values(ascending=False)
        print("Porcentagem de nulos por modalidade:")
        print(nulos_por_modalidade.head(10))

        # Contagem absoluta de nulos por modalidade
        contagem_nulos_modalidade = df[df['nomeTipo'].isnull()].groupby('nomeModalidadeCompra').size().sort_values(ascending=False)
        print("\nQuantidade de nulos por modalidade (top 10):")
        print(contagem_nulos_modalidade.head(10))


**Análise da coluna `numeroCompra`**
- Total de nulos: 1,68%

Observações:

Conclusão: Os nulos em `numeroCompra` ocorrem de forma consistente apenas quando a modalidade é 'Não se Aplica', indicando que não há um processo de compra associado. Portanto, a ausência do número nesses casos tem motivo estruturado e não representa erro de dado. → <span style='color:green; font-weight:bold;'>manter</span>

In [ ]:
if 'numeroCompra' in df.columns:
    # 1. Verifica porcentagem de nulos de 'numeroCompra'
    nulos = df['numeroCompra'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos / total * 100).round(2)
    print(f"\nTotal de nulos: {nulos} ({percentual_nulos}%)")

    # 2. Verifica relação com 'nomeModalidadeCompra'
    if 'nomeModalidadeCompra' in df.columns:
        # 2.1 Valores mais comuns de modalidade quando numeroCompra é nulo
        valores_modalidade = df[df['numeroCompra'].isnull()]['nomeModalidadeCompra'].value_counts()
        percentual_modalidade = (valores_modalidade / nulos * 100).round(2)
        print("\nModalidades de compra quando numeroCompra é nulo:")
        resultado = pd.DataFrame({
            'Contagem': valores_modalidade,
            'Percentual (%)': percentual_modalidade
        })
        print(resultado)

        # 2.2 Verifica registro com 'Não se Aplica'
        if 'Não se Aplica' in df['nomeModalidadeCompra'].values:
            total_nsa = df[df['nomeModalidadeCompra'] == 'Não se Aplica'].shape[0]
            nsa_com_nulos = df[(df['nomeModalidadeCompra'] == 'Não se Aplica') & 
                              (df['numeroCompra'].isnull())].shape[0]

            print(f"\nDos {total_nsa} registros com modalidade 'Não se Aplica':")
            print(f"- {nsa_com_nulos} ({round(nsa_com_nulos/total_nsa*100, 2)}%) têm numeroCompra nulo")

            print(f"Dos {nulos} registros com numeroCompra nulo:")
            print(f"- {nsa_com_nulos} ({round(nsa_com_nulos/nulos*100, 2)}%) têm modalidade 'Não se Aplica'")


**Análise da coluna `nomeUnidadeRealizadoraCompra`** \
*OBS: também inclui `codigoUnidadeRealizadoraCompra`*
- Total de nulos: 0,96%

Observações:

Conclusão: Os nulos em `nomeUnidadeRealizadoraCompra` ocorrem de forma consistente apenas quando a modalidade é 'Não se Aplica', indicando que não há um processo de compra associado. Portanto, a ausência da Unidade nesses casos tem motivo estruturado e não representa erro de dado. → <span style='color:green; font-weight:bold;'>manter</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeUnidadeRealizadoraCompra' in df.columns:
    # 1. Verifica porcentagem de nulos de 'nomeTipo'
    nulos_tipo = df['nomeUnidadeRealizadoraCompra'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_tipo / total * 100).round(2)
    print(f"\nTotal de nulos em nomeUnidadeRealizadoraCompra: {nulos_tipo} ({percentual_nulos}%)")
    
    # 2. Verifica relação com 'nomeModalidadeCompra'
    if 'nomeModalidadeCompra' in df.columns:
        # 2.1 Valores mais comuns de modalidade quando nomeUnidadeRealizadoraCompra é nulo
        valores_modalidade = df[df['nomeUnidadeRealizadoraCompra'].isnull()]['nomeModalidadeCompra'].value_counts()
        percentual_modalidade = (valores_modalidade / nulos_tipo * 100).round(2)
        print("\nModalidades de compra quando nomeUnidadeRealizadoraCompra é nulo:")
        resultado = pd.DataFrame({
            'Contagem': valores_modalidade,
            'Percentual (%)': percentual_modalidade
        })
        print(resultado)

        # 2.2 Verifica registro com 'Não se Aplica'
        if 'Não se Aplica' in df['nomeModalidadeCompra'].values:
            total_nsa = df[df['nomeModalidadeCompra'] == 'Não se Aplica'].shape[0]
            nsa_com_nulos = df[(df['nomeModalidadeCompra'] == 'Não se Aplica') & 
                              (df['nomeUnidadeRealizadoraCompra'].isnull())].shape[0]

            print(f"\nDos {total_nsa} registros com modalidade 'Não se Aplica':")
            print(f"- {nsa_com_nulos} ({round(nsa_com_nulos/total_nsa*100, 2)}%) têm nomeUnidadeRealizadoraCompra nulo")

            print(f"Dos {nulos_tipo} registros com nomeUnidadeRealizadoraCompra nulo:")
            print(f"- {nsa_com_nulos} ({round(nsa_com_nulos/nulos_tipo*100, 2)}%) têm modalidade 'Não se Aplica'")

**Análise da coluna `dataVigenciaFinal`** 
- Total de nulos: 0,94%
- Modalidades de compra (`nomeModalidadeCompra`) com `dataVigenciaFinal` nulos: Inexigibilidade, Não se Aplica, Dispensa, Pregão

Observações:


Conclusão: a coluna apresenta baixa porcentagem de nulos, sendo que a maioria desses nulos é provavelmente opcional, pois ocorre em contratos com modalidade de compra Inexigibilidade ou Não se Aplica. Apenas alguns nulos aparecem em modalidades onde a vigência final é obrigatória (Dispensa e Pregão) → <span style='color:green; font-weight:bold;'>manter a coluna, criando uma flag que sinalize a confiabilidade do registro, para facilitar a análise e evitar viés.</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'dataVigenciaFinal' in df.columns:
    # 1. Verifica porcentagem de nulos de 'dataVigenciaFinal'
    nulos_tipo = df['dataVigenciaFinal'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_tipo / total * 100).round(2)
    print(f"\nTotal de nulos em dataVigenciaFinal: {nulos_tipo} ({percentual_nulos}%)")

    # 2. Verifica relação com coluna 'nomeModalidadeCompra'
    if 'nomeModalidadeCompra' in df.columns:
        # Calcula a quantidade de nulos por modalidade (ao invés da média/percentual)
        nulos_por_modalidade = df.groupby('nomeModalidadeCompra')['dataVigenciaFinal'].apply(lambda x: x.isnull().sum())
        print("\nQuantidade de nulos em 'dataVigenciaFinal' por 'nomeModalidadeCompra':")
        print(nulos_por_modalidade.sort_values(ascending=False))

**Análise da coluna `nomeCategoria`** \
*OBS: também inclui `codigoCategoria`*
- Total de nulos: 0,59%

Observações:

Conclusão: A ocorrência de nulos na coluna de categoria de contrato é baixa (0,59%), possivelmente associada a falhas de sistema ou ausência de preenchimento no momento do cadastro. Como essa proporção é pouco significativa e não compromete as análises posteriores, não será necessário criar flags de confiabilidade. Também não será realizado o preenchimento com valores padrão, a fim de evitar a introdução de viés nos dados. → <span style='color:green; font-weight:bold;'>manter</span>

In [ ]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeCategoria' in df.columns:
    # 1. Verifica porcentagem de nulos de 'nomeCategoria'
    nulos_tipo = df['nomeCategoria'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_tipo / total * 100).round(2)
    print(f"\nTotal de nulos em nomeCategoria: {nulos_tipo} ({percentual_nulos}%)")

    # 2. Verifica distribuição por órgão e modalidade
    if 'nomeOrgao' in df.columns:
        print("\nDistribuição por órgão:")
        print(df[df['nomeCategoria'].isnull()]['nomeOrgao'].value_counts())

    if 'nomeModalidadeCompra' in df.columns:
        print("\nDistribuição por modalidade:")
        print(df[df['nomeCategoria'].isnull()]['nomeModalidadeCompra'].value_counts())

**Análise da coluna `nomeRazaoSocialFornecedor`**
- Total de nulos: 0,07%

Observações:
- a coluna `nomeRazaoSocialFornecedor` tem relação direta com a coluna `niFornecedor` (que é a identificação do fornecedor) por isso, poderemos usar essa segunda coluna para preencher os nulos da primeira coluna
- não foi possível preencher um dos 8 registros nulos pois o `niFornecedor` é um CPF, e mesmo fazendo buscas, não foi encontrado o pertencente a este CPF
- para buscas de CNPJ foi usado o site https://casadosdados.com.br/

Conclusão: coluna tem relação direta com outra, o que permite preencher os nulos → <span style='color:green; font-weight:bold;'>manter e preencher nulos</span>

In [17]:
# Configuração do limite para considerar exclusão
LIMITE_NULOS = 90  

if 'nomeRazaoSocialFornecedor' in df.columns:
    # 1. Verifica porcentagem de nulos de 'nomeRazaoSocialFornecedor'
    nulos_tipo = df['nomeRazaoSocialFornecedor'].isnull().sum()
    total = len(df)
    percentual_nulos = (nulos_tipo / total * 100).round(2)
    print(f"\nTotal de nulos em nomeRazaoSocialFornecedor: {nulos_tipo} ({percentual_nulos}%)")


Total de nulos em nomeRazaoSocialFornecedor: 1 (0.01%)


In [11]:
# Exporta os fornecedores com nome nulo para preenchimento manual
fornecedores_nulos = df[df['nomeRazaoSocialFornecedor'].isna()][['niFornecedor']].drop_duplicates()

# Adiciona coluna para preenchimento e normaliza os IDs
fornecedores_nulos['niFornecedor'] = fornecedores_nulos['niFornecedor'].astype(str).str.strip()
fornecedores_nulos['nomeRazaoSocialFornecedor'] = ''  # Coluna em branco para preenchimento

# Salva em CSV
pasta_referencia = Path('../data/reference')
pasta_referencia.mkdir(exist_ok=True, parents=True)  # Cria pasta se não existir
caminho_csv = pasta_referencia / 'fornecedores_para_completar.csv'

print(f"Exportando {len(fornecedores_nulos)} fornecedores sem nome para: {caminho_csv}")
fornecedores_nulos.to_csv(caminho_csv, index=False, encoding='utf-8')
print("Preencha os nomes no arquivo CSV e execute a célula seguinte para importar.")

Exportando 8 fornecedores sem nome para: ..\data\reference\fornecedores_para_completar.csv
Preencha os nomes no arquivo CSV e execute a célula seguinte para importar.


In [16]:
# Importa os dados preenchidos manualmente
caminho_csv = Path('../data/reference/fornecedores_para_completar.csv')
if caminho_csv.exists():
    # Carrega dados preenchidos
    fornecedores_preenchidos = pd.read_csv(caminho_csv, sep=';', encoding='utf-8')

    # Filtra apenas registros efetivamente preenchidos
    fornecedores_validos = fornecedores_preenchidos[
        fornecedores_preenchidos['nomeRazaoSocialFornecedor'].notna() & 
        (fornecedores_preenchidos['nomeRazaoSocialFornecedor'] != '')
    ].copy()
    
    if len(fornecedores_validos) > 0:
        # Normaliza os identificadores com zeros à esquerda quando necessário
        fornecedores_validos['niFornecedor'] = fornecedores_validos['niFornecedor'].astype(str).str.strip()
        fornecedores_validos['niFornecedor_original'] = fornecedores_validos['niFornecedor']  # Guarda original para comparação
        fornecedores_validos['niFornecedor'] = fornecedores_validos['niFornecedor'].apply(
            lambda x: x.zfill(11) if x.isdigit() and len(x) < 11 else x
        )
        
        # Mostra registros que foram ajustados
        registros_ajustados = fornecedores_validos[
            fornecedores_validos['niFornecedor'] != fornecedores_validos['niFornecedor_original']
        ]
        if len(registros_ajustados) > 0:
            print(f"Registros com zeros adicionados à esquerda: {len(registros_ajustados)}")
            for _, row in registros_ajustados.iterrows():
                print(f"  {row['niFornecedor_original']} → {row['niFornecedor']}")
        
        # Cria dicionário para mapeamento
        mapeamento_manual = dict(zip(
            fornecedores_validos['niFornecedor'], 
            fornecedores_validos['nomeRazaoSocialFornecedor']
        ))
        
        # Aplica o preenchimento (garantindo tipos consistentes)
        df['niFornecedor_str'] = df['niFornecedor'].astype(str).str.strip()
        df['niFornecedor_str'] = df['niFornecedor_str'].apply(
            lambda x: x.zfill(11) if x.isdigit() and len(x) < 11 else x
        )
        
        # Preenche os valores nulos usando o mapeamento
        df.loc[df['nomeRazaoSocialFornecedor'].isna(), 'nomeRazaoSocialFornecedor'] = \
            df.loc[df['nomeRazaoSocialFornecedor'].isna(), 'niFornecedor_str'].map(mapeamento_manual)
            
        # Remove coluna temporária
        df.drop('niFornecedor_str', axis=1, inplace=True)
        
        # Verifica resultado
        nulos_depois = df['nomeRazaoSocialFornecedor'].isna().sum()
        print(f"Foram preenchidos {len(fornecedores_validos)} fornecedores")
        print(f"Nulos restantes: {nulos_depois}")
        
        if nulos_depois > 0:
            print("IDs que ainda estão com valores nulos:")
            print(df[df['nomeRazaoSocialFornecedor'].isna()]['niFornecedor'].tolist())
    else:
        print("Nenhum fornecedor foi preenchido no arquivo CSV")
else:
    print(f"Arquivo não encontrado: {caminho_csv}")

Registros com zeros adicionados à esquerda: 1
  9975428703 → 09975428703
Foram preenchidos 7 fornecedores
Nulos restantes: 1
IDs que ainda estão com valores nulos:
['13272527915']


In [9]:
colunas_com_nulos_para_excluir = [
    'numeroControlePncpContrato',
    'dataHoraExclusao',
    'totalDespesasAcessorias',
    'nomeSubcategoria',
    'codigoSubcategoria',
    'unidadesRequisitantes',
    'valorAcumulado',
    'nomeTipo',
    'codigoTipo',
    ]

df.drop(colunas_com_nulos_para_excluir, axis=1, inplace=True)

### TRATAMENTO GERAL DAS COLUNAS

O que fazer:
- Verificar consistência nas datas de vigência (verificar se `dataVigenciaInicial` é anterior à `dataVigenciaFinal`)
- Analisar valores monetários (veriricar se `valorParcela` * `numeroParcelas` é consistente com `valorGlobal`) -> com margem de tolerância para arredondamentos
- Verificar por códigos/IDs vazios ou inválidos (verificar se tem tamanho correto, etc)
- (Talvez) Separar colunas de data em ano, mês, e dia (mantendo as colunas originais)

In [ ]:
df.dtypes

In [15]:
pd.set_option('display.max_columns', None)
df.head(3)

,codigoOrgao,nomeOrgao,codigoUnidadeGestora,nomeUnidadeGestora,codigoUnidadeGestoraOrigemContrato,nomeUnidadeGestoraOrigemContrato,receitaDespesa,numeroContrato,codigoUnidadeRealizadoraCompra,nomeUnidadeRealizadoraCompra,numeroCompra,codigoModalidadeCompra,nomeModalidadeCompra,codigoCategoria,nomeCategoria,niFornecedor,nomeRazaoSocialFornecedor,processo,objeto,informacoesComplementares,dataVigenciaInicial,dataVigenciaFinal,valorGlobal,numeroParcelas,valorParcela,dataHoraInclusao,idCompra,contratoExcluido,data_inicio,trimestre,flag_inconsistencia_compra,flag_vigencia_final_nula,flag_vigencia_obrigatoria_ausente
0,52121,ESCRITÓRIO AVANÇADO DA OPERAÇÃO CARRO-PIPA DA ...,160454,28º BATALHAO DE CACADORES,160454,28º BATALHAO DE CACADORES,D,00003/2024,160454.0,28º BATALHAO DE CACADORES,00001/2024,7,Inexigibilidade,60.0,Serviços,49269449000149,49.269.449 EDVAN FERREIRA DO NASCIMENTO,64025.000075/2024-71,"SERVIÇO DE COLETA, TRANSPORTE E DISTRIBUIÇÃO D...",NaN,2024-01-01,2024-04-30,57606.44,4,14401.61,2024-02-20 14:46:03,16045407000012024,False,2024-01-01,1,False,False,False
1,26407,"INST.FED.DE EDUC.,CIENC.E TEC.GOIANO",158124,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",158124,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",D,2024NE000042,158124.0,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",00003/2023,7,Inexigibilidade,60.0,Serviços,20320503000151,FUNDACAO ARTHUR BERNARDES,23217.000106.2022-97,PAGAMENTO DE ANUIDADE (2024) DO PROGRAMA DE AN...,15812407000032023 - UASG Minuta: 158124,2024-01-26,2024-12-31,1218.77,1,1218.77,2024-07-01 10:33:34,15812407000032023,False,2024-01-26,1,False,False,False
2,5604,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,925068,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,925068,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,D,2024NE038305,925068.0,PMSP - SUBPREFEITURA CAPELA DO SOCORRO,90007/2024,6,Dispensa,60.0,Compras,30960599000176,30.960.599 HELAINE RODRIGUES ROCHA LIMA,6057.2024/0000408-5,FORFORO / INSETICIDA,NaN,2024-03-18,2024-04-18,631.58,1,631.58,2024-07-24 12:01:39,92506806900072024,False,2024-03-18,1,False,False,False


1 - Ajeitando o tipo de dados das colunas e removendo a hora das colunas de Vigência 👇

In [ ]:
# 1. Coluna de ID/código para string
colunas_id = [
    'codigoOrgao',
    'codigoUnidadeGestora',
    'codigoUnidadeGestoraOrigemContrato',
    'codigoUnidadeRealizadoraCompra',
    'codigoCategoria'
]
for col in colunas_id:
    if col in df.columns:
        # Converte para string mantendo zeros à esquerda
        df[col] = df[col].astype(str)
        print(f"- Coluna '{col}' convertida para string (object)")

# 2. Colunas de data para datetime
colunas_data = [
    'dataVigenciaInicial',
    'dataVigenciaFinal',
    'dataHoraInclusao'
]
for col in colunas_data:
    if col in df.columns:
        # Converte para datetime, travando valores inválidos com NaT
        df[col] = pd.to_datetime(df[col], errors='coerce')
        # Remove componentes de hora se existirem (mantém apenas data)
        if 'dataVigencia' in col: # apenas para colunas de vigência
            df[col] = df[col].dt.normalize()
        print(f"- Coluna '{col}' convertida para datetime")

# 3. Verificação do tipo de dados atualizados
print("\n=== TIPOS DE DADOS ATUALIZADOS ===")
for col, tipo in df.dtypes.items():
    print(f"- Coluna '{col}': {tipo}")

- Coluna codigoOrgao convertida para string (object)
- Coluna codigoUnidadeGestora convertida para string (object)
- Coluna codigoUnidadeGestoraOrigemContrato convertida para string (object)
- Coluna codigoUnidadeRealizadoraCompra convertida para string (object)
- Coluna codigoCategoria convertida para string (object)
- Coluna dataVigenciaInicial convertida para datetime
- Coluna dataVigenciaFinal convertida para datetime
- Coluna dataHoraInclusao convertida para datetime

=== TIPOS DE DADOS ATUALIZADOS ===
codigoOrgao: object
nomeOrgao: object
codigoUnidadeGestora: object
nomeUnidadeGestora: object
codigoUnidadeGestoraOrigemContrato: object
nomeUnidadeGestoraOrigemContrato: object
receitaDespesa: object
numeroContrato: object
codigoUnidadeRealizadoraCompra: object
nomeUnidadeRealizadoraCompra: object
numeroCompra: object
codigoModalidadeCompra: object
nomeModalidadeCompra: object
codigoCategoria: object
nomeCategoria: object
niFornecedor: object
nomeRazaoSocialFornecedor: object
proce

---

2 - Limpezas básicas do Dataframe 👇

2.1 - Padronização de strings (remoção de espaços extras e uniformização de casos)

In [19]:
colunas_string = df.select_dtypes(include=['object']).columns
for col in colunas_string:
    # Ignora colunas que são IDs/códigos
    if not ('codigo' in col.lower() or 'numero' in col.lower() or 'id' in col.lower() or 'ni' in col.lower()):
        df[col] = df[col].str.strip().str.title() if not pd.isna(df[col]).all() else df[col]
        print(f"Padronizado strings em: {col}")

Padronizado strings em: nomeOrgao
Padronizado strings em: receitaDespesa
Padronizado strings em: nomeCategoria
Padronizado strings em: nomeRazaoSocialFornecedor
Padronizado strings em: processo
Padronizado strings em: objeto
Padronizado strings em: informacoesComplementares


2.3 - Verificação e tratamento de duplicatas

OBS: necessário criar flag para sinalizar as duplicatas -> incerteza se essas duplicatas são erros do sistema ou se são propositais (múltiplos registros enviados).

In [ ]:
# Verifica total de duplicatas
duplicatas = df.duplicated().sum()
print(f"Total de registros duplicados: {duplicatas} ({duplicatas/len(df)*100:.2f}%)")

# Se houver duplicatas, mostra apenas alguns exemplos
if duplicatas > 0:
    # Identifica as linhas duplicadas (mantendo todas as ocorrências)
    linhas_duplicadas = df[df.duplicated(keep=False)]
    
    # Ordenar para agrupar duplicatas
    linhas_duplicadas = linhas_duplicadas.sort_values(by='numeroContrato')
    
    # Mostra apenas algumas colunas importantes e limita a 10 exemplos
    colunas_principais = ['numeroContrato', 'nomeOrgao', 'nomeRazaoSocialFornecedor', 'valorGlobal', 'objeto', 'dataVigenciaInicial', 'dataHoraInclusao']
    print("\nExemplos de duplicatas:")
    display(linhas_duplicadas[colunas_principais].head(10))

2.4 - Verificação de outliers em colunas numéricas (incluindo valores negativos) -> inclui colunas `valorGlobal` e `valorParcela`

In [17]:
colunas_valor = [col for col in df.columns if 'valor' in col.lower()]
for col in colunas_valor:
    if col in df.columns:
        # Detecta valores negativos (possivelmente errados)
        negativos = (df[col] < 0).sum()
        if negativos > 0:
            print(f"AVISO: {negativos} valores negativos encontrados em '{col}'")
            # Decisão: converter para positivo, ou marcar como nulo

        # Detecta outliers extremos (usando IQR)
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 3 * IQR
        limite_superior = Q3 + 3 * IQR
        outliers = ((df[col] < limite_inferior) | (df[col] > limite_superior)).sum()
        print(f"Outliers em '{col}': {outliers} registros ({outliers/len(df)*100:.2f}%)")

Outliers em 'valorGlobal': 1224 registros (10.20%)
Outliers em 'valorParcela': 1329 registros (11.07%)


In [10]:
len(df.columns)

33

### CRIAÇÃO DE FLAGS

Colunas usadas para basear essa flag: `numeroCompra`, `nomeUnidadeRealizadoraCompra` e `nomeModalidadeCompra`

Explicação:
- Quando a modalidade de compra é "Não se Aplica" dá a entender que não existe uma compra, o problema é que, em alguns registros que contém o "Não se Aplica" possui valor nas colunas `numeroCompra` e `nomeUnidadeRealizadoraCompra` -> pode significar uma inconsistência nos dados (pois se não existe compra, não existe numeração e unidade) -> isso se confirma mais ainda pelo fato de quando tem nulos nessas duas colunas, a modalidade de compra sempre está como não se aplica

Ação: 
- Criar uma flag de inconsistência de compras sinalizando inconsistência em registros com "Não se Aplica" em que possuem valores nas colunas `numeroCompra` e `nomeUnidadeRealizadoraCompra`

Criação de flags para inconsistência de compras 👇

In [2]:
# Criação da flag de inconsistência de compras
df['flag_inconsistencia_compra'] = False  # Valor padrão

# Marca casos onde modalidade é "Não se Aplica" mas possui dados de compra (numeroCompra e nomeUnidadeRealizadoraCompra)
df.loc[(df['nomeModalidadeCompra'] == 'Não se Aplica') & 
       (df['numeroCompra'].notna() | df['nomeUnidadeRealizadoraCompra'].notna()),
       'flag_inconsistencia_compra'] = True

print(f"Registros com inconsistência de compra: {df['flag_inconsistencia_compra'].sum()}")

Registros com inconsistência de compra: 173


Colunas usadas para basear essa flag: `dataVigenciaFinal` e `nomeModalidadeCompra`

Explicação:
- A coluna apresenta baixa porcentagem de nulos, sendo que a maioria desses nulos é provavelmente opcional, pois ocorre em contratos com modalidade de compra Inexigibilidade ou Não se Aplica. Apenas alguns nulos aparecem em modalidades onde a vigência final é obrigatória (Dispensa e Pregão)

Ação: 
- Criar uma flag que sinalize a confiabilidade do registro, para facilitar a análise e evitar viés.


Criação de flags para confiabilidade da data de vigência 👇

In [4]:
# Flag para data de vigência final ausente
df['flag_vigencia_final_nula'] = df['dataVigenciaFinal'].isna()

# Flag específica para vigência ausente em modalidades onde seria obrigatória
modalidades_obrigatorias = ['Dispensa', 'Pregão']  # Modalidades onde vigência é mandatória
df['flag_vigencia_obrigatoria_ausente'] = False
df.loc[(df['dataVigenciaFinal'].isna()) & 
       (df['nomeModalidadeCompra'].isin(modalidades_obrigatorias)),
       'flag_vigencia_obrigatoria_ausente'] = True

Flag para duplicatas

Explicação:
- Depois de análise geral, foi identificado algumas duplicatas. Como se trata de um sistema de contratos gorvenamentais, é arriscado deduzir se um contrato duplicado é um erro ou proposital (múltiplos registros). Também não existe nada na documentação falando sobre isso. Para evitar possível perda de dados, a melhor solução é criar uma flag de sinalização.

Ação: 
- Criar uma flag que sinalize registros duplicados

In [32]:
df['flag_duplicata'] = df.duplicated(keep='first')

# Exibe estatísticas das duplicatas
duplicatas = df['flag_duplicata'].sum()
print(f"Total de registros marcados como duplicatas: {duplicatas} ({duplicatas/len(df):.2%})")
print(f"Total de contratos com mais de uma ocorrência: {df[df['flag_duplicata']]['numeroContrato'].nunique()}")

Total de registros marcados como duplicatas: 29 (0.24%)
Total de contratos com mais de uma ocorrência: 29


In [33]:
df.columns

Index(['codigoOrgao', 'nomeOrgao', 'codigoUnidadeGestora',
       'nomeUnidadeGestora', 'codigoUnidadeGestoraOrigemContrato',
       'nomeUnidadeGestoraOrigemContrato', 'receitaDespesa', 'numeroContrato',
       'codigoUnidadeRealizadoraCompra', 'nomeUnidadeRealizadoraCompra',
       'numeroCompra', 'codigoModalidadeCompra', 'nomeModalidadeCompra',
       'codigoCategoria', 'nomeCategoria', 'niFornecedor',
       'nomeRazaoSocialFornecedor', 'processo', 'objeto',
       'informacoesComplementares', 'dataVigenciaInicial', 'dataVigenciaFinal',
       'valorGlobal', 'numeroParcelas', 'valorParcela', 'dataHoraInclusao',
       'idCompra', 'contratoExcluido', 'data_inicio', 'trimestre',
       'flag_inconsistencia_compra', 'flag_vigencia_final_nula',
       'flag_vigencia_obrigatoria_ausente', 'flag_duplicata'],
      dtype='object')